### Script to read parquet files in Windows
> * Install pyspark
> * Install findspark
> * Start this script with usual jupyter notebook (Do not start with pyspark since we create the spark context in the script)

This script imports each parquet file as a spark dataframe and queries the ENCNTR_ID of interest according to the Banner_encounters.csv, then saveas it as a pandas dataframe. File name of each \*.parquet file is preserved except the extension changes to \*.pkl 

In [1]:
import findspark
import os

fileDir = os.path.dirname('__file__')
findspark.init()

import pyspark
from pyspark import SparkContext, SparkConf
# Create a directory called 'spark-warehouse' in your current working directory and run the below line
conf = SparkConf().set("spark.sql.warehouse.dir","file:///c:/Users/310248864/pedAKI/src/pedAKI_predictor/spark-warehouse")

from pyspark.sql import SparkSession

sc = pyspark.SparkContext(conf=conf)

spark = SparkSession.builder.appName("PythonSQL").config("spark.some.config.option", "some-value").getOrCreate()

In [2]:
sc._conf.set('spark.driver.memory', '20g')
sc._conf.set('spark.driver.maxResultSize', '20g')

In [4]:
pq_file = 'part-r-00866-addacd71-2d03-488a-8811-4d31f35f6a75.gz.parquet'
sk_df = spark.read.load(pq_file)

In [3]:
import glob
import pandas as pd
import gc
import numpy as np

In [4]:
encounter_df = pd.read_csv('./csv_Banner/Banner_encounters.csv')
ceid_list = encounter_df.encntr_id.tolist()

In [5]:
def filtEncounter(spark_df, ceid_list):
    pd_df = spark_df[spark_df.ENCNTR_ID.isin(ceid_list)].toPandas()
    return pd_df

In [6]:
pqDir = [x[0] for x in os.walk('D:/BannerData/160701_no_strings_has_duplicates/')]
pqDir = pqDir[1:]
for srcDir in pqDir:
    desDir = srcDir.replace('BannerData', 'BannerData_pandas')
    if not os.path.exists(desDir):
        os.makedirs(desDir)    
    pq_files = glob.glob(os.path.join(srcDir, '*.parquet'))
    for pq_file in pq_files:
        fname = pq_file.replace('BannerData', 'BannerData_pandas')
        fname = fname.replace('parquet','pkl')
        fname = fname.replace('.gz','')
        
        sk_df = spark.read.load(pq_file)
        print('loaded {}'.format(pq_file))
        pd_df = filtEncounter(sk_df, ceid_list)        
#         sk_df = sk_df.toPandas()
#         print('converted {}'.format(fname))
        
        pd_df.to_pickle(fname)
        print('saved {}'.format(fname))
        gc.collect()
    print('Complete extracting {} directory'.format(srcDir[-18:]))

loaded D:/BannerData/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00866-addacd71-2d03-488a-8811-4d31f35f6a75.gz.parquet
saved D:/BannerData_pandas/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00866-addacd71-2d03-488a-8811-4d31f35f6a75.pkl
loaded D:/BannerData/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00867-addacd71-2d03-488a-8811-4d31f35f6a75.gz.parquet
saved D:/BannerData_pandas/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00867-addacd71-2d03-488a-8811-4d31f35f6a75.pkl
loaded D:/BannerData/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00868-addacd71-2d03-488a-8811-4d31f35f6a75.gz.parquet
saved D:/BannerData_pandas/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00868-addacd71-2d03-488a-8811-4d31f35f6a75.pkl
loaded D:/BannerData/160701_no_strings_has_duplicates/enc_prefix_0=pfx00\part-r-00869-addacd71-2d03-488a-8811-4d31f35f6a75.gz.parquet
saved D:/BannerData_pandas/160701_no_strings_has_duplicates/enc_p